In [93]:
import json
import os
import pandas as pd
import pymysql

In [190]:
def read_atributes(file):

    atributos = {}
#     remove = ['nFLows']

    with open(file, 'r') as signature_file:
        signature = json.load(signature_file)
        cardinalidade = signature['metadata']['cardinality']
        for step in signature['signature']:
            for atribute in signature['signature'][step]:
#                 if atribute not in remove:
                atributos.setdefault(atribute, [signature['signature'][step][atribute]['value_min'],signature['signature'][step][atribute]['value_max']])

    return atributos, cardinalidade

In [191]:
sig_path = '/home/amanda/Desktop/dos/'

In [192]:
files = [os.path.join(sig_path, f) for f in os.listdir(sig_path) if os.path.isfile(os.path.join(sig_path, f))]

In [208]:
def query(atributos, cardinalidade, tabela):
    contar = ['nPortSrc', 'nPortDst', 'nAddrSrc', 'nAddrDst', 'nProto', 'nFLows']
    remove = ['nFLows']
    dic = {'srcport': 'L4_SRC_PORT',
            'nPortSrc': 'L4_SRC_PORT',
            'destport': 'L4_DST_PORT',
            'nPortDst': 'L4_DST_PORT',
            'nAddrSrc': 'IPV4_SRC_ADDR',
            'nAddrDst': 'IPV4_DST_ADDR',
            'proto': 'PROTOCOL',
            'nProto': 'PROTOCOL',
            'nBytesSrc': 'IN_BYTES',
            'nBytesDst': 'OUT_BYTES',
            'nPktsSrc': 'IN_PKTS',
            'nPktsDst': 'OUT_PKTS'
            }
    
    prefixo = 'select count(*) as nFLows'
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in contar and atrib_name not in remove:
            prefixo += ', count(distinct ' + dic[atrib_name] + ') as ' + str(atrib_name)
        
    sql = prefixo + ' from ' + tabela + ' where '
    statement = ''
    cont = 0
    queries = []
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name not in contar:
            if cont != 0 and cont < len(atributos.items()) - 1:
                statement += ' AND '
            statement += '(' + dic[atrib_name] + ' between ' + str(atrib_value[0]) + ' AND ' + str(atrib_value[1]) + ')'
            
        cont = cont + 1
        
    minimo = 1
    for interval in range(1000,51000,1000):
        if cardinalidade == '1-N':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR;'
        elif cardinalidade == 'N-1':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_DST_ADDR;'
        else:
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR, IPV4_DST_ADDR;'
        minimo = interval+1
        queries.append(sql + statement + sufixo)
#         print(sql + statement + sufixo)

    return queries

In [209]:
def consultar(query):
    db = pymysql.connect("200.145.216.190","root","1nstalation","ntopng")
    cursor = db.cursor()
    try:
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()
        db.close()
        return columns, results
    except:
        print ("Error: unable to fecth data")
        return [], ()
        db.close()

In [235]:
queries = []
vp = 0
vn = 0
fp = 0
fn = 0

total = 50 * len(files) * 2

for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos, cardinalidade = read_atributes(file)
    queries = query(atributos, cardinalidade, 'dos_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro positivo
        for coluna in colunas:
            for valor in df[coluna].values:
                if not(atributos[coluna][0] <= valor <= atributos[coluna][1]):
                    # É um falso negativo
                    classificador = 0
        
        if (classificador):
            vp = vp + 1
        else:
            fn = fn + 1
    
    queries = query(atributos, cardinalidade, 'leg_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro negativo
        for coluna in colunas:
            for valor in df[coluna].values:
                if atributos[coluna][0] <= valor <= atributos[coluna][1]:
                    # É um falso positivo
                    classificador = 0
                    
        if (classificador):
            vn = vn + 1
        else:
            fp = fp + 1
        
    
print('\nVerdadeiro Positivo: ' + str(vp))
print('\nFalso Negativo: ' + str(fn))
print('\nFalso Positivo: ' + str(fp))
print('\nVerdadeiro Negativo: ' + str(vn))



Assinatura: /home/amanda/Desktop/dos/dos4.sig


Assinatura: /home/amanda/Desktop/dos/dos13.sig


Assinatura: /home/amanda/Desktop/dos/dos5.sig


Assinatura: /home/amanda/Desktop/dos/dos1.sig


Assinatura: /home/amanda/Desktop/dos/dos3.sig


Assinatura: /home/amanda/Desktop/dos/dos7.sig


Assinatura: /home/amanda/Desktop/dos/dos9.sig


Assinatura: /home/amanda/Desktop/dos/dos11.sig


Assinatura: /home/amanda/Desktop/dos/dos10.sig


Assinatura: /home/amanda/Desktop/dos/dos8.sig


Assinatura: /home/amanda/Desktop/dos/dos12.sig


Assinatura: /home/amanda/Desktop/dos/dos2.sig


Assinatura: /home/amanda/Desktop/dos/dos6.sig

Verdadeiro Positivo: 112

Falso Negativo: 538

Falso Positivo: 5

Verdadeiro Negativo: 645
